Implementation of fine-tuned ResNet50V2. 94.76% accuracy on training data. 

**Import Relevant Libraries**

In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import tqdm
from keras.preprocessing import image

**Grabbing Data**

In [ ]:
train_dir = '../input/dataset/train'
test_dir = '../input/dataset/test/'

In [ ]:
df_train = pd.read_csv('/Users/ari/devel/python_projects/kaggle_projects/holiday_post_detection/dataset/train.csv')

In [ ]:
df_train.head()

In [ ]:
df_train['Class'].value_counts()

**Data Preprocessing**

In [ ]:
target_size = (256,256)
batch_size = 64

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

train_set = train_datagen.flow_from_dataframe(dataframe=df_train, directory=train_dir, x_col='Image', y_col='Class', batch_size=batch_size, seed=10, shuffle=True, class_mode='categorical', target_size=target_size)



**Base Model**

In [ ]:
base_model = tf.keras.applications.ResNet50V2(input_shape=(256,256,3), include_top=False)

**Finding Layer To Fine-Tune On**

In [ ]:
base_model.trainable = True
base_model.summary()

In [ ]:
tuning_layer_name = 'conv5_block1_preact_bn'
tuning_layer = base_model.get_layer(tuning_layer_name)
tuning_index = base_model.layers.index(tuning_layer)

for layer in base_model.layers[:tuning_index]:
    layer.trainable = False

In [ ]:
base_model.summary()

**Training Of Model**

In [ ]:
model = tf.keras.Sequential([
    base_model, tf.keras.layers.GlobalAveragePooling2D(), tf.keras.layers.Dense(6, activation='softmax')
])
lr = 0.001
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=lr), metrics=['accuracy'])

In [ ]:
history = model.fit(train_set, epochs=4)

**Making Submission CSV**

In [ ]:
classes = train_set.class_indices

In [ ]:
classes = dict((v,k) for k,v in classes.items())
classes = [classes[k] for k in classes]

In [ ]:
Image = []
prediction = []

for i, file in enumerate(os.listdir(test_dir)):
    Image.append(file)
    file = test_dir + file

    img = image.load_img(file, target_size=(256,256,3)) 
    img = image.img_to_array(img)
    img = img/255
    pred = model.predict(img.reshape(1,256,256,3))

    prediction.append(classes[np.argmax(pred[0])])

In [ ]:
submission=pd.DataFrame((zip(Image, prediction)),columns=['Images','Classes'])
submission

Upvote if this helped you please. Thanks for looking at this notebook!